<h1>Fuzzy joins with <code>merge_ordered</code> and <code>merge_asof</code></h1>

<br>

In [77]:
# libraries
import pandas as pd
import numpy as np

# import
def csv_to_df(file_path):
    df = pd.read_csv(file_path)
    return df

# data
lgn = csv_to_df('Data\data_logins.csv')
lgn['date_event'] = lgn['date_event'].astype('datetime64[ns]')
lgn['last_lgn'] = lgn['date_event']
lgn

,user_id,session_id,date_event,last_lgn
0,cbd-10827,308c29bd-8303,2022-05-01,2022-05-01
1,cbd-10827,169c25bd-2382,2022-05-06,2022-05-06
2,cbd-10827,655c26bd-1616,2022-05-07,2022-05-07
3,cbd-10827,277c67bd-5257,2022-05-13,2022-05-13
4,cbd-10827,906c88bd-1331,2022-05-16,2022-05-16
5,cbd-10827,516c27bd-8546,2022-05-22,2022-05-22
6,cbd-10827,631c89bd-3164,2022-05-24,2022-05-24
7,cbd-10827,245c19bd-9272,2022-05-31,2022-05-31


<hr>

<br>

<h2><font color='black'><code>merge_ordered</code></font></h2>

<h3><font color='grey'>Ordering time series & filling in blanks</font></h3>

<p>Joining with time series data will often result in <code>NaN</code> values being returned, e.g. if a user logs in every day but does not purchase then the purchase field will contain <code>NaN</code>s. The <code>merge_ordered</code> function allows missing observations to be filled with useful data where the join has been successful, e.g. filling <code>NaN</code> with the most recent purchase.</p>

<ul>
    <li><b><code>d</code></b> - Dataframe of dates from <code>2022-05-01</code> to <code>2022-05-31</code>.</li>
    <br>
    <li><b><code>lgn_m</code></b> - Result of <code>merge_ordered</code> between <code>d</code> and <code>lgn</code>. This builds a timeline of logins for the specified user, where values are returned for the days where there was a login.</li>
    <br>
    <li><b><code>fill_method='ffill'</code></b> - A "forward fill" is specified to populate <code>NaN</code>s with the last recorded value, e.g. fields <code>user_id</code>, <code>session_id</code>, and <code>last_lgn</code> will retrun data from the most recent login.</li>
    <br>
    <li><b><code>idle</code></b> - Idle analysis can now be completed to show how many days the account has been idle.</li>
</ul>

<br>

In [78]:
# date range
date_begin = np.datetime64('2022-05-01')
date_end = np.datetime64('2022-05-31')
d = pd.DataFrame({'date_event':np.arange(date_begin, date_end+1)})

# merge_ordered
lgn_m = pd.merge_ordered(d, lgn, on='date_event', fill_method='ffill')

# idle
lgn_m['idle_days'] = lgn_m['date_event'] - lgn_m['last_lgn']
lgn_m['idle_int'] = lgn_m['idle_days'].dt.days

# show
lgn_m.head()

,date_event,user_id,session_id,last_lgn,idle_days,idle_int
0,2022-05-01,cbd-10827,308c29bd-8303,2022-05-01,0 days,0
1,2022-05-02,cbd-10827,308c29bd-8303,2022-05-01,1 days,1
2,2022-05-03,cbd-10827,308c29bd-8303,2022-05-01,2 days,2
3,2022-05-04,cbd-10827,308c29bd-8303,2022-05-01,3 days,3
4,2022-05-05,cbd-10827,308c29bd-8303,2022-05-01,4 days,4


<hr>

<br>

<h2><font color='black'><code>merge_asof</code></font></h2>

<h3><font color='grey'>Fuzzy matches</font></h3>

<p>For time series data that does not perfectly join, the <code>merge_asof</code> function is helpful. This allows the application of logic to support a join, resulting in a fuzzy but useful match, e.g. joining login and purchasing activity to their nearest corresponding observation.</p>

<ul>
    <li><b><code>lgn</code></b> - Dataframe of user logins.</li>
    <br>
    <li><b><code>prch</code></b> - Dataframe of user purchases.</li>
    <br>
    <li><b><code>merge_asof</code></b> - By default matches login and purchase dates to the nearest counterpart, resulting in a fuzzy but useful alignment of login and purchase activity. Specifying <code>direction='nearest/forwrad/backward'</code> dictates the nature of the fuzzy match.</li>
</ul>

<br>

In [93]:
# logins
lgn = csv_to_df('Data\data_logins.csv')
lgn['date_event'] = lgn['date_event'].astype('datetime64[ns]')

# purchases
prch = csv_to_df('Data\data_purchases.csv')
prch['date_event'] = prch['date_event'].astype('datetime64[ns]')
prch['date_purchase'] = prch['date_event']
prch

,user_id,order_id,date_event,date_purchase
0,cbd-10827,#3321,2022-05-05,2022-05-05
1,cbd-10827,#8692,2022-05-11,2022-05-11
2,cbd-10827,#8864,2022-05-17,2022-05-17
3,cbd-10827,#9285,2022-05-28,2022-05-28


In [99]:
# login vs last purchase
lgn_prch = pd.merge_asof(lgn, prch, on='date_event', direction='backward', suffixes=('', '_prch'))
cols = ['user_id', 'date_event', 'date_purchase', 'order_id']
lgn_prch = lgn_prch[cols]
lgn_prch

,user_id,date_event,date_purchase,order_id
0,cbd-10827,2022-05-01,NaT,NaN
1,cbd-10827,2022-05-06,2022-05-05,#3321
2,cbd-10827,2022-05-07,2022-05-05,#3321
3,cbd-10827,2022-05-13,2022-05-11,#8692
4,cbd-10827,2022-05-16,2022-05-11,#8692
5,cbd-10827,2022-05-22,2022-05-17,#8864
6,cbd-10827,2022-05-24,2022-05-17,#8864
7,cbd-10827,2022-05-31,2022-05-28,#9285
